# Azure ML - Environments

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [9]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [10]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_name = "azure-ml-workspace-Python-SDK"

ml_client = MLClient(DefaultAzureCredential(), AZURE_SUSCRIPION_ID, AZURE_ML_RESOURCE_GRPU_ID, workspace_name)

## Environmnets

Hemos visto cómo crear `Compute Instance`s y `Compute Cluster`s, pero a la hora de entrenar un modelo necesitamos tener ciertas librerías instaladas. Para ello podemos crear entornos. La forma de hacerlo en Azure ML es mediante imágenes `Docker` o entornos de `Conda`.

Para mejorar la portabilidad normalemente se usan entornos `Conda` dentro de entornos `Docker`.

![Azure ML Environments](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/azureml-environment.webp)

#### Crear un `Environment` desde la interfaz gráfica

Para crear un nuevo `Environment` nos vamos a nuestro workspace y en el menú de la izquierda seleccionamos `Environments`, dentro de `Assets`. Vamos a la pestaña `Custom envronments` y le damos a `Create`, a partir de aquí

 * Seleccionamos un nombre, en mi caso le voy a poner `cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_GUI`
 * Le pongo la descripción `Environment cuda 11.8.0 - cudnn8 devel - ubuntu 22.04 - transformers - GUI`
 * En `Select environment source` seleccionamos `Create a new docker context`

Pulsamos en `Next`. Ahora tenemos que escribir el archivo `Dockerfile` que queremos usar. En mi caso voy a usar el siguiente

```dockerfile
FROM nvcr.io/nvidia/cuda:11.8.0-cudnn8-devel-ubuntu22.04

RUN apt-get update \
    && apt-get install -y git git-lfs python3 python3-pip \
    && curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash \
    && git lfs install \
    && rm cuda-keyring_1.0-1_all.deb \
    && ln -s /usr/bin/python3 /usr/bin/python \
    && rm -rf /var/lib/apt/lists/*

RUN pip3 install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

RUN pip3 install --no-cache-dir -U transformers accelerate datasets evaluate

WORKDIR /text-classification
```

Pulsamos en `Next` y en `Create` y ya tenemos nuestro entorno creado

### Crear un `Environment` con el CLI de Azure ML

Para crear un entorno, primero creamos el archivo de configuración del entorno de conda

In [11]:
%%writefile azure_environment_docker/dockerfile
FROM nvcr.io/nvidia/cuda:11.8.0-cudnn8-devel-ubuntu22.04

RUN apt-get update \
    && apt-get install -y git git-lfs python3 python3-pip \
    && curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash \
    && git lfs install \
    && rm cuda-keyring_1.0-1_all.deb \
    && ln -s /usr/bin/python3 /usr/bin/python \
    && rm -rf /var/lib/apt/lists/*

RUN pip3 install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

RUN pip3 install --no-cache-dir -U transformers accelerate datasets evaluate

WORKDIR /text-classification

Writing azure_environment_docker/dockerfile


Ahora creamos el entorno con el siguiente comando

In [ ]:
!GROUP='rg-azure-ml' && \
WORKSPACE='azure-ml-workspace-CLI' && \
NAME="cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_CLI" && \
DOCKER_FILE_PATH="./azure_environment_docker" && \
DOCKER_FILE="dockerfile" && \
az ml environment create --name $NAME --resource-group $GROUP --workspace-name $WORKSPACE --build-context $DOCKER_FILE_PATH --dockerfile-path $DOCKER_FILE

Uploading azure_CLI_docker (0.0 MBs): 100%|█| 605/605 [00:00<00:00, 16623.35it/s


{
  "build": {
    "dockerfile_path": "dockerfile",
    "path": "https://azuremlwstorage989ac8f2e.blob.core.windows.net/azureml-blobstore-824e3c4b-1e0b-44f9-b53b-4b14844feb97/LocalUpload/da9736b376135ad63aefe4607821f044/azure_CLI_docker/"
  },
  "creation_context": {
    "created_at": "2024-11-06T12:58:20.029673+00:00",
    "created_by": "Maximo Fernandez Nuñez",
    "created_by_type": "User",
    "last_modified_at": "2024-11-06T12:58:20.029673+00:00",
    "last_modified_by": "Maximo Fernandez Nuñez",
    "last_modified_by_type": "User"
  },
  "id": "azureml:/subscriptions/40018f20-6173-46c6-9434-ce9df3089dce/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-CLI/environments/cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_CLI/versions/3",
  "name": "cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_CLI",
  "os_type": "linux",
  "resourceGroup": "rg-azu

### Crear un `Environment` con el SDK de Python

Con la documentación de Azure no he sido capaz de crear un entorno con el SDK de Python usando un dockerfile, de modo que lo creo a mano y lo llamo `cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_python_`